<img src="img/banner.png" alt="Deparatemento de Ingeniería de Sistemas y Computación, Universidad de los Andes">

# Indexación de una Imagen - Sentinel 2

**Introducción**

Una vez se tiene el cubo funcionando y con la adición de por lo menos un producto, el siguiente paso es precisamente ingresar imágenes al cubo. Para esto, se tiene un proceso llamado **indexación**, el cual recibe una imagen, la almacena y georeferencia en la posición que le corresponde según el sistema de coordenadas geográficas que tenemos.

La indexación de imágenes consiste en:
1. Indicarle al cubo de datos dónde están imágenes de un sensor determinado en nuestro sistema de archivos.
2. Entregarle al cubo de datos los **metadatos específicos** de la imagen, para facilitar el proceso de búsqueda.
3. Asociar cada imagen de interés al producto de Sentinel 2 que ya ha sido definido con anterioridad.

**Objetivo**

El objetivo de este notebook es ilustrar el proceso de indexar (hacer que el cubo las conozca) imágenes (de un producto conocido por el cubo) al cubo y verificar luego que la imagen fue correctamente indexada.

Este proceso debe realizarse para cada imagen que se quiera ingresar en el cubo.

**Proceso para una imagen**

1. Descargar imágenes Sentinel 2
2. Descomprimir las imágenes
3. Generar los metadatos de la imagen
4. Adicionar la imagen al índice de imágenes del cubo de datos

## 1. Descargar imágenes Sentinel 2

___

Para trabajar con las imágenes, lo primero es tenerlas disponibles... Los proveedores de imnágenes generalmente proveen un servicio de búsqueda y descarga de las imágenes de interés (áreas, fechas, costos, ...). Las imágenes se estudian y se verifica si son válidas para el análisis requerido y luego finalmente se descargan (previo pago, cuando son imágenes con costo...)

Para este curso, vamos a trabajar con imágenes gratuitas provenientes de los sensores Sentinel-2 y ya se han descargado dos de ellas que nos sirven para mostrar las funcionalidades del cubo de datos.

Estas imágenes están disponibles [aquí](https://drive.google.com/drive/folders/1eNWlKCcia3FsnhFA7k61m3pnkFp3OiF6?usp=sharing). 

Para mantener nuestras imágenes organizadas, las pondremos en el siguiente directorio `/vagrant/indexed_storage/S2_MSI_L2A/`, dentro del directorio donde instalamos nuestro cubo (D:\DataCube\)

Como el cubo está en una máquina virtual Linux, alojada en una máquina física Windows, vamos a hacer la verificación de si tenemos disponibles las imágenes en los dos sistemas operacionales

**Verificación en Windows**
Navegue al directorio donde instaló el cubo (D:\DataCube) y ahí al directorio \vagrant\indexed_storage\S2_MSI_L2A\. Debe encontrar:
- Dos archivos .zip con las imágenes descargadas
- Un archivo .yaml que contiene la definición del producto Sentinel-2
- Un archivo .py, utilizado en el proceso a continuación

**Verificación en el cubo**
Verifique, ahora en el cubo, que las imágenes que descargó anteriormente estén en el directorio `/vagrant/indexed_storage/S2_MSI_L2A/`. (**Nota** Para esto se utiliza el comando `ls` de Linux)

In [2]:
!ls /vagrant/indexed_storage/S2_MSI_L2A

S2A_MSIL2A_20210130T151701_N0214_R125_T18NYM_20210130T174015.SAFE
S2A_MSIL2A_20210130T151701_N0214_R125_T18NYM_20210130T174015.zip
S2B_MSIL2A_20210115T151709_N0214_R125_T18NYM_20210115T174459.zip
sen2cor_ard.yaml
sen2cor_prepare.py


Debería haber obtenido una salida similar a la que se muestra a continuación, donde se muestran los archivos comprimidos de las imágenes. 

```sh
S2A_MSIL2A_20210130T151701_N0214_R125_T18NYM_20210130T174015.zip
S2B_MSIL2A_20210115T151709_N0214_R125_T18NYM_20210115T174459.zip
sen2cor_ard.yaml
sen2cor_prepare.py
```
En caso contrario, repita el proceso o busque dónde, en su sistema de archivos (descargas, eventualmente), quedaron las imágenes descargadas y muévalas al sitio indicado

## 2. Descomprimir una imagen

___

Seleccione uno de los archivos `.zip` que ha descargado y configure la variable `image_file_name` con el identificador de imagen como se muestra en el siguiente ejemplo.

In [3]:
# Set image_file_name with the image identifier (do not use the .zip extension)
image_file_name = 'S2A_MSIL2A_20210130T151701_N0214_R125_T18NYM_20210130T174015'

**Use el siguiente comando para descomprimir la imagen seleccionada en el directorio `/vagrant/indexed_storage/S2_MSI_L2A/`**

In [4]:
image_zip_path = f'/vagrant/indexed_storage/S2_MSI_L2A/{image_file_name}.zip'
image_dst_path = '/vagrant/indexed_storage/S2_MSI_L2A/'

!unzip -q -o $image_zip_path -d $image_dst_path

Use el siguiente comando para validar que la imagen ha sido descomprimida. La imagen descomprimida debe en una carpeta con sufijo `.SAFE`, que va precedido por el identificador de la imagen.

In [5]:
!ls /vagrant/indexed_storage/S2_MSI_L2A

S2A_MSIL2A_20210130T151701_N0214_R125_T18NYM_20210130T174015.SAFE
S2A_MSIL2A_20210130T151701_N0214_R125_T18NYM_20210130T174015.zip
S2B_MSIL2A_20210115T151709_N0214_R125_T18NYM_20210115T174459.zip
sen2cor_ard.yaml
sen2cor_prepare.py


## 3. Generar los metadatos de una imagen

___

Las imágenes que descargamos de diferentes plataformas como Copernicus o USGS Earth Explorer tienen archivos de metadatos específicos de las imágenes descargadas, pero estos metadatos no fueron generados para ser usados por herramientas como el cubo de datos. Es por esto que se requiere generar, para cada imagen, un archivo de metadatos adicional para que la imagen pueda ser interpretada por el cubo de datos. 

La generación de estos metadatos adicionales se realiza mediante **scripts de generación o preparación de metadatos**, generados por la comunidad y el equipo de desarrollo del cubo de datos. Es claro que hay un script de preparacion para cada sensor remoto.

Para este ejercicio, usaremos el script de generación de metadatos para imágenes de Sentinel 2, que se encuentra disponible en el  ambiente de desarrollo del cubo de datos, `/vagrant/indexed_storage/S2_MSI_L2A/sen2cor_prepare.py`(tomado de [aquí](https://github.com/dfnino10/ODC_Sentinel)).

Para generar los metadatos de la imágen descomprimida anteriormente, ejecute el siguiente comando:

In [6]:
# Path to the metadata generation script for sentinel 2 images
metadata_prepare_script_file_path = '/vagrant/indexed_storage/S2_MSI_L2A/sen2cor_prepare.py'
# Path to the unziped image
unziped_image_path = f'/vagrant/indexed_storage/S2_MSI_L2A/{image_file_name}.SAFE'
metadata_dst_file_path = f'/vagrant/indexed_storage/S2_MSI_L2A/{image_file_name}.SAFE'

# Execute the metadata generation script
!python3 $metadata_prepare_script_file_path  $unziped_image_path --output $metadata_dst_file_path

2021-07-23 01:36:56,140 INFO Processing /vagrant/indexed_storage/S2_MSI_L2A/S2A_MSIL2A_20210130T151701_N0214_R125_T18NYM_20210130T174015.SAFE/MTD_MSIL2A.xml
Level-2A S2MSI2A 2021-01-30T17:40:15.000000Z
2021-07-23 01:36:56,309 INFO Writing 1 dataset(s) into /vagrant/indexed_storage/S2_MSI_L2A/S2A_MSIL2A_20210130T151701_N0214_R125_T18NYM_20210130T174015.SAFE/S2A_MSIL2A_20210130T151701_N0214_R125_T18NYM_20210130T174015.SAFE.yaml


Compruebe, con el siguiente comando, si el archivo de metadatos **.SAFE.yaml** se generó para la imagen descomprimida

In [7]:
!ls -l $unziped_image_path

total 172
drwxrwxrwx 1 1000 1000     0 Jan 30 19:57 AUX_DATA
drwxrwxrwx 1 1000 1000     0 Jan 30 19:56 DATASTRIP
drwxrwxrwx 1 1000 1000     0 Jan 30 19:56 GRANULE
drwxrwxrwx 1 1000 1000  4096 Jul 23 01:36 HTML
-rwxrwxrwx 1 1000 1000 18601 Jan 30 19:57 INSPIRE.xml
-rwxrwxrwx 1 1000 1000 53047 Jan 30 19:57 MTD_MSIL2A.xml
-rwxrwxrwx 1 1000 1000  3776 Jul 23 01:36 S2A_MSIL2A_20210130T151701_N0214_R125_T18NYM_20210130T174015.SAFE.yaml
-rwxrwxrwx 1 1000 1000 91675 Jan 30 19:58 manifest.safe
drwxrwxrwx 1 1000 1000     0 Jul 23 01:36 rep_info


Para ver los metadatos de la imagen en proceso se usa el comando `cat`. Note que los metadatos contienen información especifica de cada imagen como la fecha en que fué capturada `datetime: '2021-01-15T17:44:59.000000Z'`, la ruta donde se encuentran las imágenes de las bandas en el sistema de archivos, entre otros. Finalmente, en los metadatos generados el script indica de forma explicita a qué producto del cubo de datos será asociada la imagen.

```sh
product:
  name: s2_sen2cor_ard_granule_EO3
```

Por supuesto, un producto con el identificador `s2_sen2cor_ard_granule_EO3` debe haber sido adicionado con anterioridad.

In [8]:
# Path to the image generated metadata
image_metadata_file_path = f'/vagrant/indexed_storage/S2_MSI_L2A/{image_file_name}.SAFE/{image_file_name}.SAFE.yaml'

!cat $image_metadata_file_path

id: fecb258c-968a-417e-8685-845f919db032
$schema: https://schemas.opendatacube.org/dataset
product:
  name: s2_sen2cor_ard_granule_EO3
crs: EPSG:32618
grids:
  default:
    shape:
    - 5490
    - 5490
    transform:
    - 20.0
    - 0.0
    - 699960.0
    - 0.0
    - -20.0
    - 700020.0
    - 0.0
    - 0.0
    - 1.0
  10m_res:
    shape:
    - 10980
    - 10980
    transform:
    - 10.0
    - 0.0
    - 699960.0
    - 0.0
    - -10.0
    - 700020.0
    - 0.0
    - 0.0
    - 1.0
  60m_res:
    shape:
    - 1830
    - 1830
    transform:
    - 60.0
    - 0.0
    - 699960.0
    - 0.0
    - -60.0
    - 700020.0
    - 0.0
    - 0.0
    - 1.0
measurements:
  B01_60m:
    grid: 60m_res
    path: /vagrant/indexed_storage/S2_MSI_L2A/S2A_MSIL2A_20210130T151701_N0214_R125_T18NYM_20210130T174015.SAFE/GRANULE/L2A_T18NYM_A029294_20210130T151704/IMG_DATA/R60m/T18NYM_20210130T151701_B01_60m.jp2
  B02_10m:
    grid: 10m_res
    path: /vagrant/indexed_storage/S2_MSI_L2A/S2A_MSIL2A_20210130T151701_N0214

## 4. Adicionar una imagen al índice de imágenes del cubo de datos

___

En este punto la imagen que hemos descomprimido y a la cual hemos generado los metadatos está lista para se indexada en el cubo de datos. Para indexar una imagen en el cubo de datos lo único que debemos hacer es entregar al cubo de datos el archivo de metadatos de la imagen, que fue generado en el paso anterior.

El comando para indexar una imagen en el cubo de datos tiene el siguiente formato:

`datacube dataset add <image_metadata_file>.yaml`

El comando anterior se encarga de tomar los metadatos específicos de la imagen e ingresarlos en la base de datos del cubo de datos. De esta forma, se facilita la búsqueda de imágenes en el cubo de datos.

Ejecute el comando a continuación para indexar la imagen en el cubo de datos.

In [9]:
# Utilizando la variable image_metadata_file_path definida en el paso 3

!datacube dataset add $image_metadata_file_path

Use el siguiente comando para comprobar que se ha indexado la imagen correctamente. Note que el cubo de datos muestra los metadatos obtenidos de la imagen. 

In [10]:
!datacube dataset search

id: fecb258c-968a-417e-8685-845f919db032
product: s2_sen2cor_ard_granule_EO3
status: active
locations:
- file:///vagrant/indexed_storage/S2_MSI_L2A/S2A_MSIL2A_20210130T151701_N0214_R125_T18NYM_20210130T174015.SAFE/S2A_MSIL2A_20210130T151701_N0214_R125_T18NYM_20210130T174015.SAFE.yaml
fields:
    creation_time: null
    dataset_maturity: final
    format: JPEG2000
    instrument: MSI
    label: null
    lat: {begin: 5.3333943012481395, end: 6.329895366800824}
    lon: {begin: -73.19553295981784, end: -72.20042267994468}
    platform: Sentinel-2A
    product_family: ard
    region_code: null
    time: {begin: '2021-01-30T17:40:15+00:00', end: '2021-01-30T17:40:15+00:00'}


## Repita los pasos 2 y con la otra imagen descargada.
Es cambiar el valor de la variable y volver a ejecutar los pasos 2 y 3...

**No la indexe todavía. En los primeros ejercicios con el cubo se trabaja con una sola imagen, pues la idea es entender el cubo, cómo representa la información y cómo se puede trabajar con una de ellas.
En su momento, le indicaremos que indexe la segunda imagen**

### References

1. [Indexing Data](https://datacube-core.readthedocs.io/en/latest/ops/indexing.html#indexing-data)